In [1]:
import os
import json
import pandas as pd

from zerobnl import CoSim

sim = CoSim()

/home/pablo/Projects/zerobnl/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
data_power_grid_folder = "PowerGridData"

loads = pd.DataFrame(json.load(open(os.path.join(data_power_grid_folder, 'load.json'))))
loads.index = map(int, loads.index)
loads.head()

,name,bus,p_kw,q_kvar,const_z_percent,const_i_percent,sn_kva,scaling,in_service,type
0,LoadR1,2,10.0,1.0,0.0,0.0,None,1.0,True,None
1,LoadR11,12,10.0,1.0,0.0,0.0,None,1.0,True,None
10,LoadC14,37,10.0,1.0,0.0,0.0,None,1.0,True,None
11,LoadC17,40,10.0,1.0,0.0,0.0,None,1.0,True,None
12,LoadC18,41,10.0,1.0,0.0,0.0,None,1.0,True,None


In [3]:
set_attrs = [("load/{}/p_kw".format(load), "kw") for load in loads.name]
set_attrs += [("load/{}/q_kvar".format(load), "kvar") for load in loads.name]

sim.create_meta_model("MetaGrid", set_attrs, [])
sim.create_environment("EnvGrid", "wrapper_grid.py", "Dockerfile_grid")

files = [os.path.join(data_power_grid_folder, f) for f in os.listdir(data_power_grid_folder)]
sim.add_node("Grid", "MetaGrid", "EnvGrid", files=files)

In [4]:
sim.create_meta_model("MetaLoad", [], [("p_kw", "kw"), ("q_kvar", "kvar")])
sim.create_environment("EnvLoad", "wrapper_load.py", "Dockerfile_load")

for load in loads.name:
    sim.add_node(load, "MetaLoad", "EnvLoad", init_values={"loc": 5.0, "scale": 0.5})

In [5]:
for load in loads.name:
    sim.add_link(load, "p_kw", "Grid", "load/{}/p_kw".format(load))
    sim.add_link(load, "q_kvar", "Grid", "load/{}/q_kvar".format(load))

In [6]:
sim.create_sequence([[load for load in loads.name], ["Grid"]])
sim.set_time_unit("minutes")
sim.create_steps([60] * 6)

In [7]:
sim.run()

INFO :: Simulation finished in 0 min and 25 sec


In [8]:
sim.connect_to_results_db()
sim.get_list_of_available_results()

,IN/OUT,Node,Attribute
0,IN,Grid,load/LoadC19/q_kvar
1,OUT,LoadC20,p_kw
2,IN,Grid,load/LoadC13/p_kw
3,OUT,LoadC19,p_kw
4,OUT,LoadR18,p_kw
5,IN,Grid,load/LoadI2/q_kvar
6,X,Grid,ext_grid/Feeder/p_kw
7,OUT,LoadR1,p_kw
8,IN,Grid,load/LoadC12/q_kvar
9,IN,Grid,load/LoadR16/p_kw


In [9]:
for key, value in sim.get_results_by_pattern("X*Grid*").items():
    print(key)
    print(value)

INFO :: Matching results: Grid - ext_grid/Feeder/p_kw
INFO :: Matching results: Grid - ext_grid/Feeder/q_kvar


X||Grid||ext_grid/Feeder/p_kw
2000-01-01 01:00:00   -80.471186
2000-01-01 02:00:00   -81.969109
2000-01-01 03:00:00   -81.619285
2000-01-01 04:00:00   -78.006729
2000-01-01 05:00:00   -76.138977
2000-01-01 06:00:00   -79.025460
dtype: float64
X||Grid||ext_grid/Feeder/q_kvar
2000-01-01 01:00:00   -8.232899
2000-01-01 02:00:00   -8.390987
2000-01-01 03:00:00   -8.396106
2000-01-01 04:00:00   -7.972152
2000-01-01 05:00:00   -7.762996
2000-01-01 06:00:00   -8.087487
dtype: float64
